In [1]:
# @title Installation
!pip install ipdb # debug
!pip install ipython-autotime  # timming
!pip install optuna  # hyperparaeters

In [8]:
# @title Imports

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

import ipdb

import optuna

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.3 ms


In [10]:
# seq = ['seq_'+str(i) for i in range(100)]
# X_train=pd.read_csv('./data/Xtr_mat100.csv', sep=' ', names = seq) #we use this dataset to train our model
# X_test=pd.read_csv('./data/Xte_mat100.csv', sep=' ', names = seq) #we will use this data set later to validate our model

# Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model

time: 2.05 ms


In [37]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

# X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
# X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

time: 26.8 ms


In [12]:
# X_test.head()

time: 1.47 ms


In [13]:
X_train.tail()

,Id,seq
1995,1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996,1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997,1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998,1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...
1999,1999,TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...


time: 15.8 ms


In [14]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)
time: 2.41 ms


In [15]:
# train.seq.apply(lambda x : len(x))

time: 594 µs


In [16]:
X_train.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


time: 20.3 ms


In [17]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

d = {'A': 0, 'C':1, 'G':2, 'T':2}

time: 2.05 ms


In [19]:
101-3+1

99

time: 3.15 ms


In [26]:
# X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])

def number_of_split(X_train, X_test, Y_train, n=3):
    for i in range(0, 101-n+1, 1):
    #     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    #     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])

    #     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    #     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]]

        X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+n])
        X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+n])
        
    X = X_train.drop(['seq', 'Id'], axis=1)
    X_t = X_test.drop(['seq', 'Id'], axis=1)
    y = Y_train.Bound
    return X, X_t, y


X, X_t, y = number_of_split((X_train, X_test, Y_train, n=3)) 
# X = X_train.drop(['seq', 'Id'], axis=1)
# X_t = X_test.drop(['seq', 'Id'], axis=1)
# y = Y_train.Bound

time: 381 ms


In [27]:
X.head()

,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,seq_8,seq_9,seq_10,seq_11,seq_12,seq_13,seq_14,seq_15,seq_16,seq_17,seq_18,seq_19,seq_20,seq_21,seq_22,seq_23,seq_24,seq_25,seq_26,seq_27,seq_28,seq_29,seq_30,seq_31,seq_32,seq_33,seq_34,seq_35,seq_36,seq_37,seq_38,seq_39,...,seq_61,seq_62,seq_63,seq_64,seq_65,seq_66,seq_67,seq_68,seq_69,seq_70,seq_71,seq_72,seq_73,seq_74,seq_75,seq_76,seq_77,seq_78,seq_79,seq_80,seq_81,seq_82,seq_83,seq_84,seq_85,seq_86,seq_87,seq_88,seq_89,seq_90,seq_91,seq_92,seq_93,seq_94,seq_95,seq_96,seq_97,seq_98,seq_99,seq_100
0,GAG,AGG,GGG,GGG,GGC,GCT,CTG,TGG,GGG,GGG,GGA,GAG,AGG,GGG,GGG,GGG,GGC,GCT,CTG,TGG,GGC,GCC,CCC,CCA,CAG,AGA,GAG,AGG,GGC,GCA,CAC,ACC,CCA,CAG,AGA,GAC,ACT,CTC,TCT,CTG,...,TGG,GGG,GGG,GGC,GCT,CTG,TGC,GCC,CCC,CCT,CTG,TGC,GCC,CCA,CAC,ACC,CCT,CTG,TGC,GCT,CTG,TGG,GGC,GCC,CCG,CGC,GCT,CTC,TCC,CCT,CTG,TGG,GGT,GTG,TGG,GGC,GCA,CAG,A,G
1,CGG,GGC,GCC,CCT,CTG,TGG,GGG,GGG,GGG,GGC,GCC,CCA,CAC,ACA,CAT,ATG,TGT,GTG,TGA,GAG,AGT,GTG,TGC,GCT,CTT,TTA,TAC,ACC,CCT,CTG,TGT,GTG,TGT,GTG,TGG,GGG,GGG,GGA,GAT,ATG,...,TCA,CAG,AGG,GGC,GCA,CAG,AGC,GCA,CAG,AGC,GCT,CTG,TGG,GGG,GGT,GTA,TAC,ACT,CTA,TAA,AAA,AAG,AGG,GGG,GGG,GGC,GCC,CCT,CTG,TGT,GTC,TCC,CCA,CAG,AGG,GGT,GTG,TGG,G,G
2,GAC,ACA,CAA,AAC,ACG,CGC,GCC,CCG,CGC,GCT,CTG,TGT,GTC,TCA,CAG,AGC,GCC,CCG,CGC,GCC,CCT,CTT,TTC,TCG,CGA,GAC,ACT,CTC,TCA,CAC,ACC,CCT,CTG,TGG,GGG,GGA,GAG,AGG,GGT,GTG,...,TAC,ACC,CCT,CTG,TGG,GGG,GGA,GAA,AAT,ATC,TCT,CTG,TGG,GGG,GGC,GCA,CAT,ATC,TCC,CCC,CCG,CGG,GGG,GGC,GCT,CTC,TCC,CCC,CCC,CCG,CGA,GAA,AAG,AGT,GTG,TGC,GCG,CGT,G,T
3,GCC,CCT,CTC,TCC,CCC,CCT,CTT,TTG,TGG,GGC,GCA,CAC,ACC,CCA,CAC,ACG,CGG,GGG,GGA,GAG,AGA,GAC,ACC,CCA,CAG,AGT,GTT,TTT,TTT,TTG,TGG,GGA,GAG,AGG,GGG,GGG,GGC,GCG,CGG,GGG,...,CAC,ACT,CTG,TGT,GTC,TCA,CAG,AGG,GGA,GAA,AAG,AGG,GGC,GCT,CTG,TGA,GAA,AAG,AGT,GTT,TTT,TTC,TCG,CGG,GGG,GGG,GGC,GCA,CAG,AGA,GAG,AGT,GTG,TGC,GCT,CTA,TAA,AAA,A,A
4,GCA,CAC,ACT,CTA,TAC,ACT,CTA,TAC,ACA,CAC,ACC,CCC,CCA,CAT,ATT,TTG,TGC,GCT,CTG,TGT,GTA,TAA,AAT,ATA,TAG,AGT,GTA,TAA,AAG,AGT,GTG,TGC,GCC,CCG,CGG,GGT,GTG,TGC,GCC,CCT,...,GTC,TCC,CCT,CTG,TGG,GGG,GGA,GAA,AAA,AAT,ATA,TAC,ACA,CAC,ACA,CAC,ACT,CTC,TCA,CAC,ACA,CAC,ACT,CTT,TTG,TGT,GTG,TGT,GTA,TAC,ACT,CTG,TGT,GTT,TTA,TAC,ACC,CCC,C,C


time: 227 ms


## What about counting number of different caracters 

In [145]:
# X.loc[200].iloc[98]='o'
# X.loc[200].iloc[98]

time: 691 µs


In [10]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',')

def spectrum_kernal(X_train, y, X_test, n=3, encoder=4, one_hot = True, normalise = False):
    
    d = {'A': 0.3, 'C':0.4, 'G':0.5, 'T':0.6}
    
    for i in range(0, 101-n+1, 1):
        X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+n])
        X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+n])
        
        X_train['seq_'+str(i)] = X_train['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        X_test['seq_'+str(i)] = X_test['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        
        
        
    X = X_train.drop(['seq', 'Id'], axis=1)
    X_t = X_test.drop(['seq', 'Id'], axis=1)
    y = Y_train.Bound
    
#     print(f'Train: \n{X.tail()}\n -----------------------\n')
#     print(f'Test: \n {X_t.tail()}')

    if one_hot:
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

        X_cross = X.values
        X_t = X_t.values

        y_cross = y.values


        X_cross = onehot_encoder.fit_transform(X)
        X_t_enc = onehot_encoder.fit_transform(X_t)
        
    elif normalise:
        scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
        scaler.fit(X)
        
        X_cross = scaler.transform(X)
        X_t_enc = scaler.transform(X_t)
    else :
        
        X_cross = X.values
        X_t_enc = X_t.values

    return X_cross, y, X_t_enc

X, y, X_t = spectrum_kernal(X_train, Y_train, X_test, n=5, encoder=4, one_hot = False, normalise = True)

Train: 
      seq_0  seq_1  seq_2  seq_3  seq_4  ...  seq_92  seq_93  seq_94  seq_95  seq_96
1995  743.2  799.6  814.0  817.6  716.0  ...   701.2   686.8   478.4   528.8   746.0
1996  551.6  547.2  443.6  724.8  795.2  ...   520.0   539.2   544.0   442.8   622.4
1997  759.2  496.4  533.2  747.2  493.6  ...   786.0   810.4   714.0   690.0   581.6
1998  622.4  564.8  755.2  700.4  789.2  ...   764.4   805.2   712.8   587.2   556.0
1999  496.8  635.6  772.8  807.2  713.2  ...   434.4   722.4   692.0   786.8   810.8

[5 rows x 97 columns]
 -----------------------

Test: 
      seq_0  seq_1  seq_2  seq_3  seq_4  ...  seq_92  seq_93  seq_94  seq_95  seq_96
995  760.4  804.0  712.4  484.8  632.8  ...   622.4   667.2   473.6   425.2   515.6
996  549.2  444.0  622.4  564.8  755.2  ...   686.4   683.2   580.0   451.6   419.6
997  601.2  457.2  421.2  412.0  409.6  ...   446.4   725.6   795.2   812.8   817.2
998  494.8  430.4  619.2  564.0  447.6  ...   790.0   709.2   791.2   811.6   612.0
999  

In [11]:
X.shape

(2000, 97)

time: 3.52 ms


In [12]:
X_t.shape

(1000, 97)

time: 2.87 ms


In [14]:
# X

time: 878 µs


In [15]:
# X = X_train.drop(['seq', 'Id'], axis=1)
# X_t = X_test.drop(['seq', 'Id'], axis=1)
# y = Y_train.Bound

time: 1.07 ms


## Models

In [10]:
# class LogisticRegressionBinary():
#     def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
#         self.lr = lr
#         self.num_iter = num_iter
#         self.batch_size = batch_size
#         self.verbose = verbose
    
#     def __add_intercept(self, X):
#         intercept = np.ones((X.shape[0], 1))
#         return np.concatenate((intercept, X), axis=1)
    
#     def __sigmoid_func(self, z):
#         return 1 / (1 + np.exp(-z))
    
#     def __loss(self, h, y):
#         return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
#     def fit(self, X, y):
#         y = self.trans_y(y)
#         X = self.__add_intercept(X)
#         self.theta = np.zeros(X.shape[1])
        
#         for i in range(self.num_iter):
#             z = np.dot(X, self.theta)
#             h = self.__sigmoid_func(z)
                        
#             rand = np.random.choice(y.size, self.batch_size).squeeze()
#             gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
#             self.theta -= self.lr * gradient
#             #print('theta and grad',self.theta.shape ,  gradient.shape )
#             if(self.verbose == True and i % 100 == 0):
#                 z = np.dot(X, self.theta)
#                 h = self.__sigmoid(z)
#                 print(f'loss: {self.__loss(h, y)} \t')
    
#     def predict_probability(self, X):
#         X = self.__add_intercept(X)
    
#         return self.__sigmoid_func(np.dot(X, self.theta))
    
#     def predict(self, X, threshold=.5):
#           return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
#     def Accuracy_check(self,X,y):
#         return np.mean(self.predict(X)==y)
    
#     def trans_y(self, y):
#         if isinstance(y, pd.Series):
#             y = y.values
#         if isinstance(y, list):
#             y = np.array(y)
#         return y

time: 12.2 ms


In [150]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 245 ms


# Kernel

In [151]:
import numpy as np

### Functions for you to fill in ###

def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix


def rbf_kernel_2(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    n, d = X.shape
    m = Y.shape[0]
    
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
    
    return kernel_matrix


def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

# def laplace(X1, X2, alpha=10):
#     return np.exp(-alpha*np.abs(X1-X2))

# def polynomial(X1, X2, d=2):
#     return (X1.dot(X2.T) +1)**d

def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def quadratic_kernel(X1, X2, power=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the quadratic kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (1 + linear_kernel(X1, X2))**power

def rbf_poly_kernel(X1, X2, sigma=10, d=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    
    return K+(X1.dot(X2.T) +1)**d

time: 156 ms


In [152]:
class ksolveRR_2():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
        self.sample_weights = sample_weights
            
    
    def fit(self):
        if self.sample_weights is not None:
            self.X *= self.sample_weights[:, None]
        
        X = self.X
        y = self.y
        lam = self.lam
        
        n, p = X.shape
        assert (len(y) == n)
        
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 52.8 ms


In [153]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
            
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 20.8 ms


In [154]:
import ipdb
# Logistic Ridge Regression (LRR)
class ksolveLRR():
    def __init__(self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        
        self.kernel = kernel
        
        self.sigma = sigma
        self.max_iter = max_iter
        self.tol = tol
        
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)

        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        K = self.kernel(X, X, self.sigma)

        # Initialize
        alpha = np.zeros(n)
        
        # Hint: Use IRLS
        for n_iter in range(self.max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*(f))
            
            alpha = ksolveRR_2(X, y, lam = 2*self.lam, \
                               sigma=self.sigma, sample_weights = w).fit().alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < self.tol:
                break  
                
                
        self.n_iter = n_iter
        self.alpha = alpha
        
        return self
    
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 86.3 ms


In [155]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'

import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

# SVM primal soft
class KernelSVM():
    def __init__(self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.C = C
        self.kernel = kernel
        self.lam = lam        
        self.sigma = sigma
        self.tol = tol
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        K = self.kernel(X, X, self.sigma)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
       # Compute support vectors and bias b
        sv = np.logical_and((self.alpha>self.tol), (self.C - self.alpha > self.tol))
        self.bias = y[sv] - K[sv].dot(self.alpha*y)
        self.bias =  self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
        
        
    def predict(self, X, threshold):
#         y_pred = self.kernel(X, self.X_).dot(self.alphas* seld.y_)
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where((K_x.dot(self.alpha * self.y) +  self.bias) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 285 ms


In [156]:
# # Logistic Ridge Regression (LRR)
# class perceptron():
#     def __init__(self, X, y, iteration=100):
#         self.theta = np.zeros(len(X))
#         self.X = X
#         self.y = y
#         self.iteration = iteration
#         self.theta_0 = 0
    
#     def fit(self):
        
#         X = self.X
#         y = self.y
        
#         n, p = X.shape
#         assert (len(y) == n)
        
#         for it in range(self.iteration):
#             for i in range(len(y)):
#                 if y[i]*(np.dot(self.theta, X[i]) + self.theta_0)<= 0 :
#                     self.theta = self.theta + (np.dot(y[i], X[i]))
#                     self.theta_0 += y[i]
    
        
#     def predict(self, X, threshold):
#         return np.where(X.dot(self.self.theta)+self.theta_0 >= threshold, 1, 0)
        
          
#     def Accuracy_check(self,X, y, threshold=.5):
#         return np.mean(self.predict(X, threshold)==y)

time: 3.12 ms


# Cross Validation

In [157]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TransformerM
# from sklearn.feature_extraction.text import TfidfTransformer

time: 2.17 ms


In [158]:
# vectorizer = CountVectorizer(max_features=2000)
# vectorizer.fit_transform(X[:10])
# # vectorizer.get_feature_names()

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

time: 1.48 ms


In [159]:
kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

# X_cross = onehot_encoder.fit_transform(X.values[:, :101])
# X_t_enc = onehot_encoder.fit_transform(X_t.values[:, :101])

# X_cross = onehot_encoder.fit_transform(X)
# X_t_enc = onehot_encoder.fit_transform(X_t)


X_cross = X.values
X_t_enc = X_t.values


# scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
# scaler.fit(X_cross)

# X_cross = scaler.transform(X_cross)

time: 29.2 ms


In [160]:
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.feature_selection import SelectKBest, chi2

time: 775 µs


In [161]:
# def feature_selector(X):
#     #sel = VarianceThreshold()
#     sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#     sel.fit_transform(X)
#     return X
# def best_feature_selector(X,y,num_features=50):
#     #print(X.shape)
#     features = SelectKBest(chi2, k=num_features).fit(X, y)
#     #print(X_new.shape)
#     return features

time: 2.83 ms


In [162]:
X.shape

(2000, 99)

time: 5.43 ms


In [163]:
# trans = best_feature_selector(X, y, 30)

time: 722 µs


In [164]:
# X = trans.transform(X)
# X_t = trans.transform(X_t)

time: 890 µs


In [165]:
X.values.shape

(2000, 99)

time: 2.16 ms


In [166]:
X_t.values.shape

(1000, 99)

time: 2.33 ms


In [167]:
X_t_enc.shape

(1000, 99)

time: 2.49 ms


In [168]:
X_cross.shape

(2000, 99)

time: 2.05 ms


In [169]:
# PRUNNING TRIAL 

# def objective_sgd(trial):
# #     c  = trial.suggest_loguniform('c', 1e-20, 30)
#     sigma  = trial.suggest_loguniform('sigma', 1e-3, 20) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
# #     kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
#     lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
# #     tol = trial.suggest_loguniform('tol', 1e-10, 1e-0)
    
#     # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
#     # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
#     # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
#     # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
# #     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
# #               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
# #     
#     kenel = rbf_kernel
#     models = {ksolveRR: 'k Ridge Reg'} 
      
# #     accuracy = []
#     for model in models:
#         accuracy = []
#         for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
#             X_train, y_train = X_cross[train_index], y_cross[train_index]
#             X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
            
            
# #             ipdb.set_trace()
#             if models[model] == 'weigh Ridge Reg':
#                 sample_weights = np.random.rand(len(y_train))
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
                
#             elif models[model] == 'k Logistic Ridge Reg':
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
#             elif models[model] == 'k Ridge Reg':
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
#             else:
#                 model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
#             model_curr.fit()
            
#             acc = model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5)
#             accuracy.append(acc)
            
#             intermediate_value = acc
#             trial.report(intermediate_value, i)
        
# #             print(f'accurracy fold {i}: {accuracy[i]}')
        
# #         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

#     return np.mean(accuracy)

# # Set up the median stopping rule as the pruning condition.
# # study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
# study = optuna.create_study(pruner=optuna.pruners.MedianPruner(), sampler=sampler, direction='maximize')

# # sampler = optuna.samplers.TPESampler()    # sampler = SimulatedAnnealingSampler()
# # study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective_sgd, n_trials=1500,show_progress_bar=True)


# trial = study.best_trial

# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

time: 7.63 ms


In [31]:
2e1

20.0

time: 2.72 ms


In [ ]:
def objective_sgd(trial):
    c  = trial.suggest_loguniform('c', 1e-20, 2e1)
    sigma  = trial.suggest_float('sigma', 1e-20, 20) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
    kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
    lam = trial.suggest_float('lam', 1e-20, 1e-1)
    tol = trial.suggest_loguniform('tol', 1e-7, 1e-0)
    
    model = trial.suggest_categorical('models', [ksolveRR , ksolveRR_2, ksolveLRR, KernelSVM])
    
    # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
    # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
    # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
    # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
    models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
              ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
#     
#     kenel = rbf_kernel
#     models = {ksolveRR : 'k Ridge Reg'} 
      
#     accuracy = []
#     for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]


#             ipdb.set_trace()
        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)

        model_curr.fit()
        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#             print(f'accurracy fold {i}: {accuracy[i]}')
        
#         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()

study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=5500,show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7fdeba31ff28> which is of type function.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7fdeba335f28> which is of type function.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for pe

[W 2020-05-29 03:24:02,451] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7fdeba31ff28> which is of type function.
[W 2020-05-29 03:24:02,455] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7fdeba335f28> which is of type function.
[W 2020-05-29 03:24:02,461] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.
[W 2020-05-29 03:24:02,464] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR_2'> which is of type type.
[W 2020-05-29 03:24:02,467] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent st

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: RuntimeWarning:

invalid value encountered in greater_equal



     pcost       dcost       gap    pres   dres
 0: -4.0684e+02 -2.3258e+00  7e+03  8e+01  6e-16
 1: -4.6021e+00 -2.3243e+00  7e+01  8e-01  4e-16
 2: -5.8307e-01 -2.1833e+00  2e+00  8e-03  5e-16
 3: -6.1885e-01 -7.2317e-01  1e-01  2e-04  7e-16
 4: -6.7327e-01 -6.7440e-01  1e-03  2e-06  3e-16
 5: -6.7381e-01 -6.7382e-01  1e-05  2e-08  2e-16
 6: -6.7382e-01 -6.7382e-01  1e-07  2e-10  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.9182e+02 -2.3133e+00  7e+03  8e+01  6e-16
 1: -4.4322e+00 -2.3118e+00  7e+01  8e-01  5e-16
 2: -5.6155e-01 -2.1686e+00  2e+00  8e-03  4e-16
 3: -5.9602e-01 -6.9974e-01  1e-01  2e-04  7e-16
 4: -6.4843e-01 -6.4955e-01  1e-03  2e-06  3e-16
 5: -6.4895e-01 -6.4896e-01  1e-05  2e-08  2e-16
 6: -6.4895e-01 -6.4895e-01  1e-07  2e-10  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.0233e+02 -2.3220e+00  7e+03  8e+01  6e-16
 1: -4.5511e+00 -2.3205e+00  7e+01  8e-01  2e-16
 2: -5.7662e-01 -2.1789e

In [30]:
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.624
Best hyperparameters: {'sigma': 0.5676751743446526, 'lam': 5.181586550863371e-06}
time: 973 µs


In [ ]:
# kenel = rbf_kernel
#     models = {ksolveRR: 'k Ridge Reg'} 
# Accuracy: 0.624
# Best hyperparameters: {'sigma': 0.5676751743446526, 'lam': 5.181586550863371e-06}
# time: 973 µs

#------------------------



In [ ]:
# Augmented data

# sigma  = trial.suggest_loguniform('sigma', 1e-20, 20)
# lam = trial.suggest_loguniform('lam', 1e-20, 1e-1)

# models = {ksolveRR: 'k Ridge Reg'} (Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.065620491225982, 'lam': 0.013374107290659768}
    # time: 49min 21s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575 # 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.119788517147901, 'lam': 1.2752298700618223e-14}, {'sigma': 4.063158315715049, 'lam': 8.30834772639223e-05}
    # time: 32min 57s

    
# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)

# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.069738272304652, 'lam': 0.053817561640154984}
    # time: 48min 12s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.001185698777986, 'lam': 1.9770379950513775e-10}
    # time: 29min 58s
    


# Original data

# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
  
# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data standardize)
    # kenel = rbf_kernel
    # Accuracy: 0.612
    # Best hyperparameters: {'sigma': 2.8925324917718167, 'lam': 1.2575244169567934e-08}
    # time: 28min 33s 
    #----
    # kenel = quadratic_kernel
    # Accuracy: EROOR LinearAlg
    # Best hyperparameters: 
    # time: 

In [31]:
#     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
#               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'

# c  = 
sigma  = 4.119788517147901
kenel = rbf_kernel
lam = 1.2752298700618223e-14
# tol = 
    
#     ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
#     ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
#     ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
#     KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel

    
models = {ksolveRR : 'k Ridge Reg'}

for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=500, tol = tol, kernel = kenel)
        
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.4875
accurracy fold 1: 0.4725
accurracy fold 2: 0.5475
accurracy fold 3: 0.495
accurracy fold 4: 0.5025

Average accuracy k Ridge Reg is : 0.501

time: 9.2 s


In [56]:
# 0.6535 = 0.68800
# 0.657 = 0.69200

time: 525 µs


In [58]:
# # Cehckinf full model
# model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
# # model = svm_primal_soft_to_qp(X_cross, y_cross, C=1)

# model.fit()

# model.Accuracy_check(X_cross, y_cross, threshold=0.5)

time: 881 µs


# Predictions

In [59]:
model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

time: 393 ms


In [60]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


time: 8.73 ms


In [61]:
sample['Bound'] = y_pred

time: 1.42 ms


In [62]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,1
998,998,1
999,999,1


time: 7.8 ms


In [63]:
sample.to_csv('./ksolveRR_6575_cv_rbf_kernel_sigma_4.119788517147901_lam_1.2752298700618223e-14.csv', index=False)

time: 3.12 ms
